# KNN Classifier without PCA

# KNN Classifier with PCA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score

In [2]:
#loading in the datasets
df_training = pd.read_csv('../data/archive/train_pca.csv')
df_testing = pd.read_csv('../data/archive/test_pca.csv')

In [3]:
#getting all of our randomized samples from the training
all_samples = []

for sample in range(0,5):
    all_samples.append(df_training.sample(n=5000, replace= True))

In [4]:
#Separating X and y from testing since this won't be used in the for loop
x_train_full = df_training.iloc[:,:-1]
y_train_full = df_training.iloc[:,-1]

x_test = df_testing.iloc[:, :-1]
y_test = df_testing.iloc[:,-1]

In [5]:
#initializing our lists so we can later store the best params that achieve best scores across metrics
accuracy_best_params = []
f1_best_params = []
precision_best_params = []


for sample in all_samples:
    
    knn = KNeighborsClassifier()
    
    #Making a grid of values we want our grid search to test to find the best parameters
    grid_values ={'n_neighbors': list(range(1,105,4)), 'weights':['uniform','distance']}
    
    knn_classifier = GridSearchCV(estimator = knn, param_grid = grid_values, cv = StratifiedKFold(n_splits = 5),
                      scoring = ['accuracy', 'f1_micro', 'precision_micro'], refit = False, verbose = 0)
    
    x_train = sample.iloc[:, :-1]
    y_train = sample.iloc[:,-1]
    
    knn_model = knn_classifier.fit(x_train, y_train)

    accuracy_best_params.append(knn_model.cv_results_['params'][ np.argmin(knn_model.cv_results_['rank_test_accuracy'])])
    f1_best_params.append(knn_model.cv_results_['params'][ np.argmin(knn_model.cv_results_['rank_test_f1_micro'])])
    precision_best_params.append(knn_model.cv_results_['params'][ np.argmin(knn_model.cv_results_['rank_test_precision_micro'])])

In [6]:
accuracy_best_params

[{'n_neighbors': 13, 'weights': 'distance'},
 {'n_neighbors': 5, 'weights': 'distance'},
 {'n_neighbors': 1, 'weights': 'uniform'},
 {'n_neighbors': 9, 'weights': 'distance'},
 {'n_neighbors': 9, 'weights': 'distance'}]

In [7]:
f1_best_params

[{'n_neighbors': 13, 'weights': 'distance'},
 {'n_neighbors': 5, 'weights': 'distance'},
 {'n_neighbors': 1, 'weights': 'uniform'},
 {'n_neighbors': 9, 'weights': 'distance'},
 {'n_neighbors': 9, 'weights': 'distance'}]

In [8]:
precision_best_params

[{'n_neighbors': 13, 'weights': 'distance'},
 {'n_neighbors': 5, 'weights': 'distance'},
 {'n_neighbors': 1, 'weights': 'uniform'},
 {'n_neighbors': 9, 'weights': 'distance'},
 {'n_neighbors': 9, 'weights': 'distance'}]

In [9]:
accuracy_test_score = []

i = 0
for param in accuracy_best_params:
    
    knn_clf = KNeighborsClassifier(n_neighbors = accuracy_best_params[i].get('n_neighbors'))
    model = knn_clf.fit(x_train_full, y_train_full)

    y_pred = model.predict(x_test)

    accuracy_test_score.append(accuracy_score(y_test, y_pred))
    
    i+=1

In [10]:
accuracy_test_score

[0.8853070919579233,
 0.8768238887003733,
 0.8517136070580251,
 0.8873430607397353,
 0.8873430607397353]

In [13]:
f1_test_score = []

i = 0
for param in f1_best_params:
    
    knn_clf = KNeighborsClassifier(n_neighbors = f1_best_params[i].get('n_neighbors'))
    model = knn_clf.fit(x_train_full, y_train_full)

    y_pred = model.predict(x_test)

    f1_test_score.append(f1_score(y_test, y_pred, average = 'micro'))
    
    i+=1

In [14]:
f1_test_score

[0.8853070919579233,
 0.8768238887003733,
 0.8517136070580251,
 0.8873430607397353,
 0.8873430607397353]

In [16]:
precision_test_score = []

i = 0
for param in precision_best_params:
    
    knn_clf = KNeighborsClassifier(n_neighbors = precision_best_params[i].get('n_neighbors'))
    model = knn_clf.fit(x_train_full, y_train_full)

    y_pred = model.predict(x_test)

    precision_test_score.append(precision_score(y_test, y_pred, average = 'micro'))
    
    i+=1

In [17]:
precision_test_score

[0.8853070919579233,
 0.8768238887003733,
 0.8517136070580251,
 0.8873430607397353,
 0.8873430607397353]